In [1]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier as sklDecisionTreeClassifier, export_graphviz
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
iris = load_iris()
X, y = iris.data, iris.target
# clf = tree.DecisionTreeClassifier()
# clf = clf.fit(X, y)
# iris.keys()
from sklearn.datasets import load_wine
wine = load_wine()
X, y = wine.data, wine.target
# X.shape, y.shape

In [2]:
# from sklearn.datasets import load_wine
# wine = load_wine()
# X, y = wine.data, wine.target
# X.shape, y.shape

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25, random_state=10)

In [4]:
import numpy as np
from collections import Counter
import random
from tree.decision_tree import DecisionTree
clf = DecisionTree(max_depth=5, criterion='gini')
clf.fit(X_train, y_train)
y_test_pred = clf.predict(X_test)
y_train_pred = clf.predict(X_train)
print(accuracy_score(y_train, y_train_pred), accuracy_score(y_test, y_test_pred))
clf = DecisionTree(max_depth=5, criterion='gini', extreme_random=True)
clf.fit(X_train, y_train)
y_test_pred = clf.predict(X_test)
y_train_pred = clf.predict(X_train)
print(accuracy_score(y_train, y_train_pred), accuracy_score(y_test, y_test_pred))
clf = sklDecisionTreeClassifier(max_depth=5, criterion='gini')
clf.fit(X_train, y_train)
y_test_pred = clf.predict(X_test)
y_train_pred = clf.predict(X_train)
print(accuracy_score(y_train, y_train_pred), accuracy_score(y_test, y_test_pred))
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=1000)
clf.fit(X_train, y_train)
y_test_pred = clf.predict(X_test)
y_train_pred = clf.predict(X_train)
print(accuracy_score(y_train, y_train_pred), accuracy_score(y_test, y_test_pred))

1.0 0.8888888888888888
0.9924812030075187 0.9555555555555556
1.0 0.8888888888888888
1.0 0.9555555555555556


In [5]:
class RandomForestBase:
    """
    A Random Forest Classifier Machine Learning algorithm.

    Parameters:
        n_estimators (int): The number of decision trees to use
        max_tree_depth (int): The maximum depth of the component decision tree
        criterion (str): The criterion for splitting - 'gini' or 'entropy'
        max_features_split (float): The maximum number of features to consider for making a decision tree split (default=number of features)
        extreme_random (bool): Whether to use randomly generated thresholds for splitting (default=False)
    """

    def __init__(self, 
        n_estimators: int=100, 
        max_tree_depth: int=20, 
        criterion: str='gini',
        max_features_split: int=None,
        extreme_random: bool=False
        ):
        self.n_estimators = n_estimators
        self.max_tree_depth = max_tree_depth
        self.criterion = criterion
        self.max_features_split = max_features_split
        self.extreme_random = extreme_random


    def generate_random_decision_trees(self, X, y, n_estimators, max_features_split, random_state=100):
        print(n_estimators)
        estimators = []
        for _ in range(n_estimators):
            features_to_skip = []
            if self.max_features_split is not None:
                # print(self.n_features, self.n_features-self.max_features_split)
                features_to_skip = list(np.random.choice(self.n_features, self.n_features-self.max_features_split, replace=False))
                # print(features_to_skip)

            clf = DecisionTree(
                max_depth=self.max_tree_depth, 
                criterion=self.criterion, 
                features_to_skip=features_to_skip,
                extreme_random=self.extreme_random
                )
            random_indices = np.random.choice(X.shape[0], X.shape[0], replace=True)
            X_sample = X[random_indices]
            y_sample = y[random_indices]
            clf.fit(X_sample, y_sample)
            estimators.append(clf)
        return estimators



class RandomForestClassifier(RandomForestBase):
    def __init__(self, 
        n_estimators: int=100, 
        max_tree_depth: int=20, 
        criterion: str='gini',
        max_features_split: int=None,
        extreme_random: bool=False
        ):
        super().__init__(n_estimators=n_estimators, max_tree_depth=max_tree_depth, 
            criterion=criterion, max_features_split=max_features_split,
            extreme_random=extreme_random
            )


    def fit(self, X: np.array, y: np.array):
        n_samples, n_features = X.shape
        assert n_samples == y.shape[0]
        assert len(y.shape) == 1 or y.shape[1] == 1

        self.n_samples, self.n_features = n_samples, n_features

        self.num_classes = len(Counter(y))

        self.estimators = self.generate_random_decision_trees(X, y, self.n_estimators, self.max_features_split)

        return self


    def predict_proba(self, X):
        y_preds = [clf.predict(X) for clf in self.estimators]
        y_preds = [np.squeeze(np.eye(self.num_classes)[a.reshape(-1)]) for a in y_preds]
        y_pred_proba = np.array(y_preds).mean(axis=0)

        return y_pred_proba


    def predict(self, X):
        y_pred_proba = self.predict_proba(X)
        y_pred = y_pred_proba.argmax(axis=1)

        return y_pred


class RandomForestRegressor(RandomForestBase):
    def __init__(self, 
        n_estimators: int=100, 
        max_tree_depth: int=20, 
        criterion: str='gini',
        max_features_split: int=None,
        extreme_random: bool=False
        ):
        super().__init__(n_estimators=n_estimators, max_tree_depth=max_tree_depth, 
            criterion=criterion, max_features_split=max_features_split,
            extreme_random=extreme_random
            )


    def fit(self, X: np.array, y: np.array):
        n_samples, n_features = X.shape
        assert n_samples == y.shape[0]
        assert len(y.shape) == 1 or y.shape[1] == 1

        self.n_samples, self.n_features = n_samples, n_features

        self.estimators = self.generate_random_decision_trees(X, y, self.n_estimators, self.max_features_split)

        return self


    def predict(self, X):
        y_preds = np.array([clf.predict(X) for clf in self.estimators]).mean(axis=0)

        return y_preds

In [6]:
clf = RandomForestClassifier(n_estimators=100, criterion='gini', extreme_random=False, max_features_split=8)
clf.fit(X_train, y_train)
y_test_pred = clf.predict(X_test)
y_train_pred = clf.predict(X_train)
print(accuracy_score(y_train, y_train_pred), accuracy_score(y_test, y_test_pred))

100
1.0 0.9111111111111111


In [7]:
from sklearn.datasets import load_diabetes
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.ensemble import RandomForestRegressor as sklRandomForestRegressor
from sklearn.tree import DecisionTreeRegressor as sklDecisionTreeRegressor
data = load_diabetes()
X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25, random_state=10)
data.keys()

dict_keys(['data', 'target', 'frame', 'DESCR', 'feature_names', 'data_filename', 'target_filename', 'data_module'])

In [8]:
clf = sklDecisionTreeRegressor(max_depth=10)
clf = sklRandomForestRegressor(max_depth=10)
clf.fit(X_train, y_train)
y_test_pred = clf.predict(X_test)
y_train_pred = clf.predict(X_train)
# accuracy_score(y_train, y_train_pred), accuracy_score(y_test, y_test_pred)
print(r2_score(y_test, y_test_pred), mean_squared_error(y_test, y_test_pred), r2_score(y_test, np.ones(y_test.shape)*np.mean(y_test)))
# dot_data = clf.export_graphviz(feature_names=data.feature_names, regression=True)
# graph = graphviz.Source(dot_data)
# graph 

0.4814179240098624 3256.690470682478 0.0


In [11]:
# clf = DecisionTree(max_depth=10, criterion='variance')
clf = RandomForestRegressor(n_estimators = 100, criterion='variance', max_tree_depth=10)
clf.fit(X_train, y_train)
y_test_pred = clf.predict(X_test)
y_train_pred = clf.predict(X_train)
# accuracy_score(y_train, y_train_pred), accuracy_score(y_test, y_test_pred)
print(r2_score(y_test, y_test_pred), mean_squared_error(y_test, y_test_pred), r2_score(y_test, np.ones(y_test.shape)*np.mean(y_test)))
# dot_data = clf.export_graphviz(feature_names=data.feature_names, regression=True)
# graph = graphviz.Source(dot_data)
# graph

100


/home/tvpc00032/miniconda3/envs/ml_scratch/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/tvpc00032/miniconda3/envs/ml_scratch/lib/python3.11/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/tvpc00032/Code/projects/ml-from-scratch/packages/tree/tree/decision_tree.py:49: RuntimeWarning: Degrees of freedom <= 0 for slice
  variance = y.var()
/home/tvpc00032/miniconda3/envs/ml_scratch/lib/python3.11/site-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/home/tvpc00032/miniconda3/envs/ml_scratch/lib/python3.11/site-packages/numpy/core/_methods.py:261: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


0.46733009232291556 3345.1619187553015 0.0
